# Advent of Code

## 2018-012-024
## 2018 024

https://adventofcode.com/2018/day/24

In [ ]:
import re
from collections import defaultdict

class Group:
    def __init__(self, team, units, hp, weaknesses, immunities, attack_damage, attack_type, initiative):
        self.team = team
        self.units = units
        self.hp = hp
        self.weaknesses = weaknesses
        self.immunities = immunities
        self.attack_damage = attack_damage
        self.attack_type = attack_type
        self.initiative = initiative

    def effective_power(self):
        return self.units * self.attack_damage

    def damage_to(self, other):
        if self.attack_type in other.immunities:
            return 0
        if self.attack_type in other.weaknesses:
            return self.effective_power() * 2
        return self.effective_power()

    def __repr__(self):
        return f"{self.team} ({self.units} units)"

def parse_input(file_path):
    with open(file_path, "r") as f:
        lines = f.read().strip().split("\n")
    
    armies = defaultdict(list)
    current_team = None
    group_pattern = re.compile(
        r"(\d+) units each with (\d+) hit points (\(.*\))? with an attack that does (\d+) (\w+) damage at initiative (\d+)"
    )
    for line in lines:
        if line.endswith(":"):
            current_team = line[:-1]
        elif line.strip():
            match = group_pattern.match(line)
            if match:
                units = int(match.group(1))
                hp = int(match.group(2))
                attack_damage = int(match.group(4))
                attack_type = match.group(5)
                initiative = int(match.group(6))
                weaknesses = []
                immunities = []

                if match.group(3):
                    attributes = match.group(3).strip("()").split("; ")
                    for attribute in attributes:
                        if attribute.startswith("weak to"):
                            weaknesses = attribute[len("weak to "):].split(", ")
                        elif attribute.startswith("immune to"):
                            immunities = attribute[len("immune to "):].split(", ")

                armies[current_team].append(
                    Group(current_team, units, hp, weaknesses, immunities, attack_damage, attack_type, initiative)
                )
    return armies

def simulate_battle(armies):
    while all(len(armies[team]) > 0 for team in armies):
        all_groups = sorted(
            [group for groups in armies.values() for group in groups],
            key=lambda g: (g.effective_power(), g.initiative),
            reverse=True,
        )
        targets = {}
        for group in all_groups:
            enemies = armies["Infection"] if group.team == "Immune System" else armies["Immune System"]
            enemies = [enemy for enemy in enemies if enemy not in targets.values() and enemy.units > 0]
            if not enemies:
                continue
            target = max(enemies, key=lambda e: (group.damage_to(e), e.effective_power(), e.initiative), default=None)
            if group.damage_to(target) > 0:
                targets[group] = target

        all_groups.sort(key=lambda g: g.initiative, reverse=True)
        for group in all_groups:
            if group.units <= 0 or group not in targets:
                continue
            target = targets[group]
            damage = group.damage_to(target)
            killed_units = damage // target.hp
            target.units = max(0, target.units - killed_units)

        for team in list(armies.keys()):
            armies[team] = [group for group in armies[team] if group.units > 0]
            if not armies[team]:
                del armies[team]

    winner_team = next(iter(armies.keys()))
    total_units = sum(group.units for group in armies[winner_team])
    return winner_team, total_units

if __name__ == "__main__":
    file_path = "part-001-sample-input.txt"
    armies = parse_input(file_path)
    winner, units = simulate_battle(armies)
    print(f"Winning army: {winner} with {units} units remaining.")

In [1]:
import re
from collections import defaultdict

class Group:
    def __init__(self, team, units, hp, weaknesses, immunities, attack_damage, attack_type, initiative):
        self.team = team
        self.units = units
        self.hp = hp
        self.weaknesses = weaknesses
        self.immunities = immunities
        self.attack_damage = attack_damage
        self.attack_type = attack_type
        self.initiative = initiative

    def effective_power(self):
        return self.units * self.attack_damage

    def damage_to(self, other):
        if self.attack_type in other.immunities:
            return 0
        if self.attack_type in other.weaknesses:
            return self.effective_power() * 2
        return self.effective_power()

    def __repr__(self):
        return f"{self.team} ({self.units} units)"

def parse_input(file_path):
    with open(file_path, "r") as f:
        lines = f.read().strip().split("\n")
    
    armies = defaultdict(list)
    current_team = None
    group_pattern = re.compile(
        r"(\d+) units each with (\d+) hit points (\(.*\))? with an attack that does (\d+) (\w+) damage at initiative (\d+)"
    )
    for line in lines:
        if line.endswith(":"):
            current_team = line[:-1]
        elif line.strip():
            match = group_pattern.match(line)
            if match:
                units = int(match.group(1))
                hp = int(match.group(2))
                attack_damage = int(match.group(4))
                attack_type = match.group(5)
                initiative = int(match.group(6))
                weaknesses = []
                immunities = []

                if match.group(3):
                    attributes = match.group(3).strip("()").split("; ")
                    for attribute in attributes:
                        if attribute.startswith("weak to"):
                            weaknesses = attribute[len("weak to "):].split(", ")
                        elif attribute.startswith("immune to"):
                            immunities = attribute[len("immune to "):].split(", ")

                armies[current_team].append(
                    Group(current_team, units, hp, weaknesses, immunities, attack_damage, attack_type, initiative)
                )
    return armies

def simulate_battle(armies):
    while all(len(armies[team]) > 0 for team in armies):
        # Sort groups by effective power and initiative for target selection
        all_groups = sorted(
            [group for groups in armies.values() for group in groups],
            key=lambda g: (-g.effective_power(), -g.initiative)
        )
        targets = {}
        for group in all_groups:
            enemies = armies["Infection"] if group.team == "Immune System" else armies["Immune System"]
            # Filter for valid targets
            enemies = [enemy for enemy in enemies if enemy.units > 0 and enemy not in targets.values()]
            if not enemies:
                continue
            # Select the best target
            target = max(
                enemies,
                key=lambda e: (group.damage_to(e), e.effective_power(), e.initiative),
                default=None
            )
            if group.damage_to(target) > 0:
                targets[group] = target

        # Sort for attack phase by initiative
        all_groups.sort(key=lambda g: -g.initiative)
        for group in all_groups:
            if group.units <= 0 or group not in targets:
                continue
            target = targets[group]
            damage = group.damage_to(target)
            killed_units = min(damage // target.hp, target.units)
            target.units -= killed_units

        # Remove groups with no units left
        for team in list(armies.keys()):
            armies[team] = [group for group in armies[team] if group.units > 0]
            if not armies[team]:
                del armies[team]

        # If no progress is made, break the loop to prevent infinite runs
        if len(targets) == 0:
            break

    # Determine the winner and remaining units
    winner_team = next(iter(armies.keys()))
    total_units = sum(group.units for group in armies[winner_team])
    return winner_team, total_units

if __name__ == "__main__":
    file_path = "input.txt"
    armies = parse_input(file_path)
    winner, units = simulate_battle(armies)
    print(f"Winning army: {winner} with {units} units remaining.")

Winning army: Infection with 4546 units remaining.


In [2]:
import re
from collections import defaultdict

class Group:
    def __init__(self, team, units, hp, weaknesses, immunities, attack_damage, attack_type, initiative):
        self.team = team
        self.units = units
        self.hp = hp
        self.weaknesses = weaknesses
        self.immunities = immunities
        self.attack_damage = attack_damage
        self.attack_type = attack_type
        self.initiative = initiative

    def effective_power(self):
        return self.units * self.attack_damage

    def damage_to(self, other):
        if self.attack_type in other.immunities:
            return 0
        if self.attack_type in other.weaknesses:
            return self.effective_power() * 2
        return self.effective_power()

    def __repr__(self):
        return f"{self.team} ({self.units} units)"

def parse_input(file_path):
    with open(file_path, "r") as f:
        lines = f.read().strip().split("\n")
    
    armies = defaultdict(list)
    current_team = None
    group_pattern = re.compile(
        r"(\d+) units each with (\d+) hit points (\(.*\))? with an attack that does (\d+) (\w+) damage at initiative (\d+)"
    )
    for line in lines:
        if line.endswith(":"):
            current_team = line[:-1]
        elif line.strip():
            match = group_pattern.match(line)
            if match:
                units = int(match.group(1))
                hp = int(match.group(2))
                attack_damage = int(match.group(4))
                attack_type = match.group(5)
                initiative = int(match.group(6))
                weaknesses = []
                immunities = []

                if match.group(3):
                    attributes = match.group(3).strip("()").split("; ")
                    for attribute in attributes:
                        if attribute.startswith("weak to"):
                            weaknesses = attribute[len("weak to "):].split(", ")
                        elif attribute.startswith("immune to"):
                            immunities = attribute[len("immune to "):].split(", ")

                armies[current_team].append(
                    Group(current_team, units, hp, weaknesses, immunities, attack_damage, attack_type, initiative)
                )
    return armies

def simulate_battle(armies):
    previous_states = set()

    while all(len(armies[team]) > 0 for team in armies):
        # Serialize state for infinite loop detection
        state = tuple((g.team, g.units, g.hp, g.attack_damage, g.initiative) for g in sorted(
            [g for groups in armies.values() for g in groups],
            key=lambda g: (g.team, g.initiative)
        ))
        if state in previous_states:
            print("Infinite loop detected. Combat ends in a draw.")
            return None, 0
        previous_states.add(state)

        # Sort groups by effective power and initiative
        all_groups = sorted(
            [group for groups in armies.values() for group in groups],
            key=lambda g: (-g.effective_power(), -g.initiative)
        )

        # Target selection phase
        targets = {}
        for group in all_groups:
            enemies = armies["Infection"] if group.team == "Immune System" else armies["Immune System"]
            enemies = [e for e in enemies if e.units > 0 and e not in targets.values()]
            if not enemies:
                continue
            target = max(
                enemies,
                key=lambda e: (group.damage_to(e), e.effective_power(), e.initiative),
                default=None
            )
            if group.damage_to(target) > 0:
                targets[group] = target

        # Attack phase
        all_groups.sort(key=lambda g: -g.initiative)
        for group in all_groups:
            if group.units <= 0 or group not in targets:
                continue
            target = targets[group]
            damage = group.damage_to(target)
            killed_units = min(damage // target.hp, target.units)
            target.units -= killed_units

        # Cleanup defeated groups
        for team in list(armies.keys()):
            armies[team] = [group for group in armies[team] if group.units > 0]
            if not armies[team]:
                del armies[team]

    winner_team = next(iter(armies.keys()))
    total_units = sum(group.units for group in armies[winner_team])
    return winner_team, total_units

if __name__ == "__main__":
    file_path = "input.txt"
    armies = parse_input(file_path)
    winner, units = simulate_battle(armies)
    print(f"Winning army: {winner} with {units} units remaining.")

Infinite loop detected. Combat ends in a draw.
Winning army: None with 0 units remaining.
